# Trabajo final HCC

In [1]:
import numpy as np
from PIL import Image
import sys
import os
import pandas as pd
import colorsys as color
import scipy.io.wavfile as wav

In [2]:
import simpleaudio as sa

In [10]:
%%capture --no-stdout
#Primero utilizamos una funcion que permite obtener los valores RGB de la imagen para cada pixel
            
def obtener_valores_rgb(path):
    imagen = Image.open(path)
    ancho, alto = imagen.size
    valores_rgb = []

    for y in range(alto):
        for x in range(ancho):
            r, g, b = imagen.getpixel((x, y))
            valores_rgb.append((r, g, b))

    return valores_rgb, ancho, alto #Extraemos la lista de valores rgb y el ancho y alto de la imagen en pixels

#Ponemos la imagen que queremos abrir
dir_actual = os.getcwd()
path = input(f'Escriba la direccion de la imagen (usted esta en {dir_actual}) ')
mayor_o_menor = input(f'Escriba si quiere utilizar la escala menor [m] o mayor [M] [Default = M] ')
nombre = input('Escriba el nombre del archivo de salida ')
try:
    valores_rgb, ancho_imagen, alto_imagen = obtener_valores_rgb(path)



    df =pd.DataFrame(valores_rgb, columns=['Red','Green','Blue']) #Convertimos la lista de tuplas a un DataFrame de pandas

    #Queremos asignar un sonido por color, por lo cual nos es conveniente pasar al sistema hls de clasificacion de color
    #Este tiene tres parametros: Matiz (Hue), Saturación(Saturation) y Luminosidad (Luminosity). Nos interesa el matiz
    matiz=[]
    saturacion=[]
    valor=[]
    for i in df.index:
        h, s, v = color.rgb_to_hsv(df['Red'][i], df['Green'][i], df['Blue'][i]) #Usamos la libreria colorsys que ya tiene la transformacion programada

        matiz.append(h)
        saturacion.append(s)
        valor.append(v)

    df['Matiz'] = matiz
    df['Saturacion'] = saturacion
    df['Valor'] = valor



    #Ahora, convertimos matiz a frecuencia con una regla de 3 simple
    df['Frecuencia del pixel']=(df['Matiz']*(2100-150)+150)  #Multiplicamos para llevar a unidades de frecuencia
    #Escalamos los matices mas cercanos a 0 a una frecuecia minima de 150 y los matices cercanos a 1 aproximadamente a la frecuencia de Do 7 (2093,01)

    #Cuando tenemos una amplia gama de frecuencias desfazadas el sonido es bastante caotico y poco agradable.
    #Es por eso que fijamos una lista de frecuencias correspondientes a las notas del acorde de Do mayor en distintas alturas.


    
    Mi3 = 164.914
    Mib3 = 155.563
    Do4 = 261.63
    Sol4 = 391.995
    Si4 = 493.883
    Sib4 = 466.164

    frecuencias_mayores=[Mi3,Do4,Mi3*2,Sol4,Si4,Do4*2,Mi3*4,Sol4*2,Si4*2,Do4*4,Mi3*8,Sol4*4,Si4*4,Do4*8]

    frecuencias_menores=[Mib3,Do4,Mib3*2,Sol4,Sib4,Do4*2,Mib3*4,Sol4*2,Sib4*2,Do4*4,Mib3*8,Sol4*4,Sib4*4,Do4*8]

    #Admito que con esto perdemos mucha informacion de los colores particulares, pero la misma puede recuperarse añadiendo mas frecuencias a la lista

    #Nuestro dataframe recorre la imagen de izquierda a derecha por renglon. Al convertirla a sonido queremos que recorra
    #todos los renglones a la vez de izquierda a derecha. Es por eso que dividimos nuestro dataframe en cada fila de pixeles
    numero_de_filas_de_pixeles=int(len(df)/ancho_imagen)
    df_filas=pd.DataFrame()
    for i in range(numero_de_filas_de_pixeles):
        df_filas[f'Fila {i}']=list(df['Frecuencia del pixel'][i*ancho_imagen:(i+1)*ancho_imagen])

    #Otro gran problema es que en una imagen de miles de pixeles de ancho y alto el costo de calcular se hace alto.
    #Por eso tomamos la simplificacion de calcular la frecuencia promedio de un conjunto de pixeles.

    pixeles_por_nota=2 #Toma los pixeles de una columna en conjuntos de N
    agrupacion_de_pixeles = numero_de_filas_de_pixeles/pixeles_por_nota
    df_cortado = np.hsplit(df_filas, agrupacion_de_pixeles)



    df_reducido=df_cortado.copy()

    for i in range(int(agrupacion_de_pixeles)):
        # Ahora calculamos el valor medio por fila y crea una nueva columna con el resultado
        filas=df_cortado[i].columns
        df_cortado[i][f'Media {i}'] = df_cortado[i].mean(axis=1)
        # Eliminamos las columnas originales y nos quedamos con este valor promedio como el representativo de esos N pixeles
        df_reducido[i] = df_cortado[i].drop(filas, axis=1)

    resultado = pd.concat(df_reducido, axis=1) #Y volvemos a unir los conjuntos de pixeles ahora reducidos
    #Asi pasamos de tener AAAxBBB pixeles que calcular a tener AAA/N x BBB pixeles que calcular

    #Ahora entran en juego las frecuencias que fijamos en "lista_de_frecuencias"

    def encontrar_mas_cercano(valor, lista):
        #Esta funcion busca la frecuencia en la lista que mas se acerque a la frecuencia del pixel
        array_lista = np.array(lista)
        indice = (np.abs(array_lista - valor)).argmin()
        return lista[indice]

    def reemplazar_cercano(valor, lista):
        #Y esta funcion se encarga de reemplazar la frecuencia original por la que mas se parecia
        mas_cercano = encontrar_mas_cercano(valor, lista)
        return mas_cercano

    #Aplicamos directamente a nuestro dataframe
    if mayor_o_menor == "m":
    
        df_reemplazado = resultado.applymap(lambda x: reemplazar_cercano(x, frecuencias_menores))
    else:
        df_reemplazado = resultado.applymap(lambda x: reemplazar_cercano(x, frecuencias_mayores))

    # Momento de hacer el sonido

    sample_rate= 44100   #La frecuencia de muestreo es el numero de muestras por segundo para crear una señal discreta de sonido
    T = .11 # Tiempo en segundos que suena cada columna de pixeles
    t = np.linspace(0, T, int(T * sample_rate), False)   #Creamos un linspace para ajustar funciones seno que representan los sonidos de cada nota


    df_notas=df_reemplazado.copy()

    audio=np.zeros(ancho_imagen*len(t)) ##Audio es un array de numpy que simpleaudio utiliza para generar el sonido
    #Nosotros lo creamos con un tamaño del numero de pixels por fila por la cantidad de puntos que definen la forma funcion de onda
    #(dado que cada pixel tiene su propia funcion de onda formada por "int(T * sample_rate)" puntos)

    for filas in df_notas.columns:
        notas=[]
        for columnas in range(ancho_imagen):
            notas.append(np.sin(df_notas[filas][columnas] * t * 2 * np.pi)) #notas sera una lista de todas las notas en una fila,
            #representada por un array de puntos que forman la longitud de onda

        audio += np.hstack(notas) #En audio sumamos el efecto de cada nota, lo que hacemos es sumar todas las notas de una misma columna
        #que, por superposicion de ondas, formaran la armonia que representa a esa columna de la imagen


    #Transformaciones que utiliza simpleaudio para pasar el sonido a 16bit, no entiendo completamente, pero formaba parte del instructivo
    audio *= 32767 / np.max(np.abs(audio))
    audio = audio.astype(np.int16)

    from scipy.io.wavfile import write

    scaled = audio
    write(f'{nombre}.wav', sample_rate, scaled)
    

    ##Finalmente reproducimos el audio igual que en los modelos de simpleaudio
    #play_obj = sa.play_buffer(audio, 1, 2, sample_rate)
    #play_obj.wait_done()
except FileNotFoundError:
    print()
    print('El archivo no existe o la ubicacion es incorrecta, intente nuevamente.')

Escriba la direccion de la imagen (usted esta en /home/tomas/Escritorio/HCC) fondo.jpg
Escriba si quiere utilizar la escala menor [m] o mayor [M] [Default = M] M
Escriba el nombre del archivo de salida fondo


In [11]:
print(audio)

[    0  5228 10168 ... 13458 13798 13305]


In [6]:
t

array([0.00000000e+00, 2.26757370e-05, 4.53514739e-05, ...,
       1.09931973e-01, 1.09954649e-01, 1.09977324e-01])